In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from finta import TA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.regularizers import l2
from sklearn.model_selection import KFold

In [2]:
# Load data
TM_path = Path("../files/TM.csv")
VIX_path = Path("../files/^VIX.csv")
FEDFUNDS_path = Path("../files/FEDFUNDS (1).csv")
SPY_index = Path("../files/SPY.csv")

In [3]:
tm_df = pd.read_csv(TM_path, index_col="Date")
fear_index_df = pd.read_csv(VIX_path, index_col="Date").rename(columns={"Close": "Fear_index"})
spy_index_df = pd.read_csv(SPY_index, index_col="Date").rename(columns={"Close": "SPY_index"})
fedfunds_df = pd.read_csv(FEDFUNDS_path, index_col="DATE")

In [4]:
# Convert index to datetime index
fedfunds_df.index = pd.to_datetime(fedfunds_df.index)

# Resample the fed_funds_df to have daily frequency and forward fill the values
fedfunds_df_monthly = fedfunds_df.resample('D').ffill()


In [5]:
# Concatenate dataframes
concatenated_df = pd.concat([tm_df, fear_index_df['Fear_index'], spy_index_df['SPY_index']], axis=1)
concatenated_df.index = pd.to_datetime(concatenated_df.index)


In [6]:
# Merge with fedfunds_df
concatenated_df = pd.merge(concatenated_df, fedfunds_df_monthly, left_index=True, right_index=True)


In [7]:
# Drop rows with NaN values
concatenated_df = concatenated_df.dropna()


In [8]:
# Shift target variable
concatenated_df['Target'] = concatenated_df['Close'].shift(-5)
concatenated_df = concatenated_df.dropna()

In [9]:
concatenated_df.head(10)

,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target
2014-04-30,108.000000,108.500000,107.669998,108.419998,100.839981,467700.0,13.41,188.309998,0.09,109.029999
2014-05-01,110.019997,110.150002,109.370003,109.730003,102.058388,340900.0,13.25,188.330002,0.09,108.150002
2014-05-02,109.989998,110.180000,109.669998,109.930000,102.244415,420500.0,12.91,188.059998,0.09,108.870003
2014-05-05,109.779999,109.930000,109.410004,109.440002,101.788673,274800.0,13.29,188.419998,0.09,109.209999
2014-05-06,109.500000,109.570000,108.720001,109.000000,101.379425,429400.0,13.80,186.779999,0.09,111.379997
2014-05-07,109.000000,109.139999,108.580002,109.029999,101.407333,319000.0,13.40,187.880005,0.09,110.430000
2014-05-08,108.500000,108.610001,107.529999,108.150002,100.588860,366500.0,13.43,187.679993,0.09,109.500000
2014-05-09,108.959999,109.379997,108.529999,108.870003,101.258522,371700.0,12.92,187.960007,0.09,109.610001
2014-05-12,108.870003,109.320000,108.599998,109.209999,101.574753,346200.0,12.23,189.789993,0.09,108.610001
2014-05-13,110.209999,111.540001,110.089996,111.379997,103.593040,564200.0,12.13,189.960007,0.09,107.669998


In [10]:
# Calculate technical indicators using finta
data = concatenated_df.copy()  # Use the existing DataFrame concatenated_df
data['MA'] = TA.SMA(data, 20)  # 20-period Simple Moving Average
data['RSI'] = TA.RSI(data, 14)  # 14-period RSI

# Calculate Bollinger Bands correctly
bb_bands = TA.BBANDS(data, 20, 2)

# Assign Bollinger Bands values to DataFrame columns
data['BB_UPPER'] = bb_bands['BB_UPPER']
data['BB_MIDDLE'] = bb_bands['BB_MIDDLE']
data['BB_LOWER'] = bb_bands['BB_LOWER']

# Convert index to datetime
data.index = pd.to_datetime(data.index)

# Display the calculated technical indicators
data.tail()


,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2024-03-18,237.580002,238.830002,237.279999,238.660004,238.660004,194100.0,14.33,512.859985,5.33,252.410004,238.4520,57.155161,252.187283,238.4520,224.716716
2024-03-19,242.679993,245.089996,242.410004,244.320007,244.320007,347400.0,13.82,515.710022,5.33,253.720001,239.3295,62.009328,252.133199,239.3295,226.525800
2024-03-20,244.679993,246.800003,244.309998,246.720001,246.720001,214900.0,13.04,520.479980,5.33,252.149994,240.3110,63.878136,252.136371,240.3110,228.485629
2024-03-21,251.449997,251.789993,250.259995,251.479996,251.479996,245500.0,12.92,522.200012,5.33,251.679993,241.1625,67.312529,253.645172,241.1625,228.679828
2024-03-22,255.000000,255.229996,253.589996,254.770004,254.770004,285400.0,13.06,521.210022,5.33,242.149994,242.1510,69.472956,255.667277,242.1510,228.634723


In [11]:
# Define features and target
X = concatenated_df.drop("Close", axis=1)
y = concatenated_df["Close"]

In [12]:
data.drop(columns=['Open', 'High', 'Low', 'Close', 'Adj Close'], inplace=True)


In [13]:
# Display the modified DataFrame
data.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2014-04-30,467700.0,13.41,188.309998,0.09,109.029999,NaN,NaN,NaN,NaN,NaN
2014-05-01,340900.0,13.25,188.330002,0.09,108.150002,NaN,100.000000,NaN,NaN,NaN
2014-05-02,420500.0,12.91,188.059998,0.09,108.870003,NaN,100.000000,NaN,NaN,NaN
2014-05-05,274800.0,13.29,188.419998,0.09,109.209999,NaN,72.857125,NaN,NaN,NaN
2014-05-06,429400.0,13.80,186.779999,0.09,111.379997,NaN,57.709400,NaN,NaN,NaN


In [14]:
data_clean = data.dropna()
data_clean.index.rename('date', inplace=True)
data_clean.to_csv('../clean_data/GM_prepared_data.csv', index=True)

In [15]:
data_clean.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
date,,,,,,,,,,
2014-05-28,259100.0,11.68,191.380005,0.09,114.480003,109.3095,55.911969,111.065231,109.3095,107.553770
2014-05-29,348500.0,11.57,192.369995,0.09,114.589996,109.4630,64.453519,111.416912,109.4630,107.509089
2014-05-30,449700.0,11.40,192.679993,0.09,115.150002,109.6320,70.717723,112.178102,109.6320,107.085898
2014-06-02,660400.0,11.58,192.899994,0.10,115.070000,109.8330,73.341083,113.029735,109.8330,106.636265
2014-06-03,593400.0,11.87,192.800003,0.10,113.870003,110.1070,76.013474,114.020872,110.1070,106.193128


In [16]:
# Define date cutoff for data split
date_cutoff = "2022-04-30"

# Split data
X_train = X[X.index <= date_cutoff]
X_test = X[X.index > date_cutoff]
y_train = y[y.index <= date_cutoff]
y_test = y[y.index > date_cutoff]

In [17]:
# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
# Define the number of folds
k = 10

# Initialize lists to store R-squared scores
train_r2_scores = []
test_r2_scores = []

# Initialize KFold
kf = KFold(n_splits=k, shuffle=True)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Example: Train with a smaller learning rate
from keras.optimizers import Adam
adam = Adam(learning_rate=0.0001)  # Adjust learning rate as needed
model.compile(optimizer=adam, loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Compile the model
model.compile(optimizer=adam, loss='mean_squared_error')

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    
    # Evaluate the model on training data
    train_predictions = model.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_r2_scores.append(train_r2)
    
    # Evaluate the model on test data
    test_predictions = model.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_r2_scores.append(test_r2)

# Calculate average R-squared scores
avg_train_r2 = np.mean(train_r2_scores)
avg_test_r2 = np.mean(test_r2_scores)

print("Average R-squared (Train):", avg_train_r2)
print("Average R-squared (Test):", avg_test_r2)

/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 327us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 344us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 566us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 362us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 544us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 476us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 328us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 495us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 348us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 319us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 503us/step
Average R-squared (Train): -37.2181815601277
Average R-squared (Test): -38.23582409232334


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize lists to store metrics for selected models
selected_train_r2 = []
selected_test_r2 = []
selected_train_mae = []
selected_train_mse = []
selected_test_mae = []
selected_test_mse = []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    # Define a new model for each fold
    model_fold = Sequential([
        Dense(32, activation='relu', input_shape=(X_train_cv.shape[1],)),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(16, activation='relu'),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(1)  # Output layer
    ])
    
    # Compile the model with Adam optimizer and mean squared error loss
    model_fold.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    # Train the model
    model_fold.fit(X_train_cv, y_train_cv, epochs=50, batch_size=32, verbose=0)
    
    # Evaluate the model on training data
    train_predictions = model_fold.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_mae = mean_absolute_error(y_train_cv, train_predictions)
    train_mse = mean_squared_error(y_train_cv, train_predictions)
    
    # Evaluate the model on test data
    test_predictions = model_fold.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_mae = mean_absolute_error(y_test_cv, test_predictions)
    test_mse = mean_squared_error(y_test_cv, test_predictions)
    
    # Append metrics to the lists
    selected_train_r2.append(train_r2)
    selected_test_r2.append(test_r2)
    selected_train_mae.append(train_mae)
    selected_train_mse.append(train_mse)
    selected_test_mae.append(test_mae)
    selected_test_mse.append(test_mse)

# Print metrics for selected models
for idx, (train_r2, test_r2, train_mae, train_mse, test_mae, test_mse) in enumerate(zip(selected_train_r2, selected_test_r2, selected_train_mae, selected_train_mse, selected_test_mae, selected_test_mse), start=1):
    print(f"Model {idx} - Train R-squared: {train_r2}, Test R-squared: {test_r2}, Train MAE: {train_mae}, Train MSE: {train_mse}, Test MAE: {test_mae}, Test MSE: {test_mse}")


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step
Model 1 - Train R-squared: 0.4788861968086989, Test R-squared: 0.4643296434547146, Train MAE: 14.724497842791743, Train MSE: 241.0399739538871, Test MAE: 15.26947540355962, Test MSE: 263.8625779725303
Model 2 - Train R-squared: 0.596296313523046, Test R-squared: 0.634493239524897, Train MAE: 13.167964055214004, Train MSE: 187.6960682278035, Test MAE: 12.695497055755917, Test MSE: 172.21381182735325
Model 3 - Train R-squared: 0.3111619865557663, Test R-squared: 0.08165408833683974, Train MAE: 17.25010955086981, Train MSE: 329.2222846248649, Test MAE: 17.12400598232035, Test MSE: 320.1618560170444
Model 4 - Train R-squared: 0.42282173492797315, Test R-squared: 0.3561986578385373, Train MAE: 15.575720496921997, Train MSE: 270.418900305462, Test MAE: 15.749280523626044, Test MSE: 282.09977565186483
Model 5 - Train R-squared: 0.44340883686650456, Test R-squared: 0.46515915381615547, Train MAE: 15.054786252867506

In [20]:
# Calculate the absolute difference between train R-squared and test R-squared values for each model
abs_diff_r2 = np.abs(np.array(selected_train_r2) - np.array(selected_test_r2))

# Find the index of the model with the smallest absolute difference
best_model_index = np.argmin(abs_diff_r2)

# Retrieve the metrics for the best model
best_train_r2 = selected_train_r2[best_model_index]
best_test_r2 = selected_test_r2[best_model_index]
best_train_mae = selected_train_mae[best_model_index]
best_train_mse = selected_train_mse[best_model_index]
best_test_mae = selected_test_mae[best_model_index]
best_test_mse = selected_test_mse[best_model_index]

# Print metrics for the best model
print(f"Best Model - Train R-squared: {best_train_r2}, Test R-squared: {best_test_r2}, Train MAE: {best_train_mae}, Train MSE: {best_train_mse}, Test MAE: {best_test_mae}, Test MSE: {best_test_mse}")


Best Model - Train R-squared: 0.6242440178503812, Test R-squared: 0.6275954223240493, Train MAE: 12.646683354693957, Train MSE: 174.5139442558589, Test MAE: 12.74901510589873, Test MSE: 174.9710081495359


In [21]:
# Filter models where both Train R-squared and Test R-squared are less than 0.96
filtered_indices = [i for i, (train_r2, test_r2) in enumerate(zip(selected_train_r2, selected_test_r2)) if train_r2 < 0.96 and test_r2 < 0.96]

# Calculate the absolute difference between train R-squared and test R-squared values for filtered models
abs_diff_r2_filtered = np.abs(np.array(selected_train_r2)[filtered_indices] - np.array(selected_test_r2)[filtered_indices])

# Find the index of the model with the smallest absolute difference among filtered models
best_model_index = filtered_indices[np.argmin(abs_diff_r2_filtered)]

# Retrieve the metrics for the best model
best_train_r2 = selected_train_r2[best_model_index]
best_test_r2 = selected_test_r2[best_model_index]
best_train_mae = selected_train_mae[best_model_index]
best_train_mse = selected_train_mse[best_model_index]
best_test_mae = selected_test_mae[best_model_index]
best_test_mse = selected_test_mse[best_model_index]

# Print metrics for the best model
print(f"Best Model - Train R-squared: {best_train_r2}, Test R-squared: {best_test_r2}, Train MAE: {best_train_mae}, Train MSE: {best_train_mse}, Test MAE: {best_test_mae}, Test MSE: {best_test_mse}")


Best Model - Train R-squared: 0.6242440178503812, Test R-squared: 0.6275954223240493, Train MAE: 12.646683354693957, Train MSE: 174.5139442558589, Test MAE: 12.74901510589873, Test MSE: 174.9710081495359


In [22]:
# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the entire dataset
model.fit(X_scaled, y, epochs=50, batch_size=32, verbose=0)

# Predict sequentially on each data point
all_predictions = model.predict(X_scaled)

# Ensure the number of predictions matches the original dataset
assert len(all_predictions) == len(X_scaled)

# Create a DataFrame to store the actual and predicted values
predictions_df = pd.DataFrame({'Actual': y, 'Predicted': all_predictions.flatten()}, index=X.index)

# Ensure index uniqueness in both the original dataset and predictions DataFrame
data_clean_unique_index = data_clean.index.drop_duplicates()
predictions_df = predictions_df.loc[data_clean_unique_index]

# Display the DataFrame
predictions_df


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step


,Actual,Predicted
date,,
2014-05-28,109.910004,82.073395
2014-05-29,111.489998,82.282333
2014-05-30,113.110001,82.858902
2014-06-02,113.949997,85.006004
2014-06-03,114.919998,84.857727
...,...,...
2024-03-18,238.660004,194.290466
2024-03-19,244.320007,197.649841
2024-03-20,246.720001,200.734528


In [23]:
predictions_df.to_csv('../predicted_data/TM_predicted_data.csv', index=True)

In [24]:
# Check for NaN values in the predictions DataFrame
nan_values = predictions_df.isnull().sum().sum()

if nan_values == 0:
    print("No NaN values found in the predictions DataFrame.")
    print(predictions_df)
else:
    print(f"Found {nan_values} NaN values in the predictions DataFrame. Please check your data or model.")


No NaN values found in the predictions DataFrame.
                Actual   Predicted
date                              
2014-05-28  109.910004   82.073395
2014-05-29  111.489998   82.282333
2014-05-30  113.110001   82.858902
2014-06-02  113.949997   85.006004
2014-06-03  114.919998   84.857727
...                ...         ...
2024-03-18  238.660004  194.290466
2024-03-19  244.320007  197.649841
2024-03-20  246.720001  200.734528
2024-03-21  251.479996  205.084351
2024-03-22  254.770004  207.008698

[2473 rows x 2 columns]


In [25]:
# Calculate the percentage difference between actual and predicted values
predictions_df['Percentage Difference (%)'] = ((predictions_df['Predicted'] - predictions_df['Actual']) / predictions_df['Actual']) * 100

# Display the DataFrame with percentage difference
predictions_df


,Actual,Predicted,Percentage Difference (%)
date,,,
2014-05-28,109.910004,82.073395,-25.326729
2014-05-29,111.489998,82.282333,-26.197565
2014-05-30,113.110001,82.858902,-26.744849
2014-06-02,113.949997,85.006004,-25.400609
2014-06-03,114.919998,84.857727,-26.159303
...,...,...,...
2024-03-18,238.660004,194.290466,-18.591107
2024-03-19,244.320007,197.649841,-19.102065
2024-03-20,246.720001,200.734528,-18.638729


In [26]:
# Calculate the absolute percentage difference for each data point
predictions_df['Abs_Percentage_Diff'] = abs((predictions_df['Actual'] - predictions_df['Predicted']) / predictions_df['Actual']) * 100

# Calculate the average percentage difference
avg_percentage_diff = predictions_df['Abs_Percentage_Diff'].mean()

print("Average Percentage Difference (%):", avg_percentage_diff)


Average Percentage Difference (%): 23.90997815636999
